In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('../../../capstone_other/files_too_large_for_github/ML_table_with_availability_clusters.csv')

In [3]:
data = data[['dock_id', 'dock_name', 'latitude', 'longitude',
           'avail_bikes', 'tot_docks', 'avail_bikes_proportion']]

In [4]:
data = data.groupby('dock_id').apply(pd.DataFrame.sample, n=1).reset_index(drop=True) 

In [5]:
data

,dock_id,dock_name,latitude,longitude,avail_bikes,tot_docks,avail_bikes_proportion
0,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,15,62,0.241935
1,119,Park Ave & St Edwards St,40.696089,-73.978034,10,19,0.526316
2,120,Lexington Ave & Classon Ave,40.686768,-73.959282,11,19,0.578947
3,143,Clinton St & Joralemon St,40.692395,-73.993379,17,24,0.708333
4,144,Nassau St & Navy St,40.698399,-73.980689,13,19,0.684211
...,...,...,...,...,...,...,...
324,3505,Lexington Ave & E 127 St,40.805726,-73.936322,4,33,0.121212
325,3506,Lexington Ave & E 120 St,40.801307,-73.939817,1,29,0.034483
326,3507,Park Ave & E 124 St,40.804555,-73.939686,1,36,0.027778
327,3509,Lenox Ave & W 115 St,40.801194,-73.950074,31,31,1.000000


In [6]:
data_low = data[data['avail_bikes_proportion'] <= 1/3]

In [7]:
data_high = data[data['avail_bikes_proportion'] >= 2/3]

In [19]:
data_low['deficit'] = round((1/3 - data_low['avail_bikes_proportion']) * data_low['tot_docks']).astype('int')

<ipython-input-19-ae1edfedde08>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_low['deficit'] = round((1/3 - data_low['avail_bikes_proportion']) * data_low['tot_docks']).astype('int')


In [20]:
data_high['surplus'] = round((data_high['avail_bikes_proportion'] - 2/3) * data_high['tot_docks']).astype('int')

<ipython-input-20-11911a3c46e5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_high['surplus'] = round((data_high['avail_bikes_proportion'] - 2/3) * data_high['tot_docks']).astype('int')


In [28]:
data_low = data_low.sort_values(by = 'deficit', ascending = False)

In [27]:
data_high = data_high.sort_values(by = 'surplus', ascending = False)

In [67]:
data_low.loc[24, 'deficit'] = 18


18

In [85]:
rebalancing_dict = {}

In [86]:
low_copy = data_low.copy()
high_copy = data_high.copy()

In [87]:
for low in low_copy.index:
    for high in high_copy.index:
        if high_copy.loc[high, 'surplus'] == 0:
            break
        stations_key = (low_copy.loc[low, 'dock_id'], high_copy.loc[high, 'dock_id'])
        change = min(low_copy.loc[low, 'deficit'], high_copy.loc[high, 'surplus'])
        low_copy.loc[low, 'deficit'] -= change
        high_copy.loc[high, 'surplus'] -= change
        if stations_key in rebalancing_dict.keys():
            rebalancing_dict[stations_key] += change
        else:
            rebalancing_dict[stations_key] = change
    if low_copy.loc[low, 'deficit'] == 0:
        break

In [88]:
rebalancing_dict

{(267, 336): 12,
 (267, 3132): 6,
 (267, 3224): 0,
 (267, 3509): 0,
 (267, 3459): 0,
 (267, 3296): 0,
 (267, 337): 0,
 (267, 439): 0,
 (267, 3365): 0,
 (267, 448): 0,
 (267, 3112): 0,
 (267, 3358): 0,
 (267, 3477): 0,
 (267, 3254): 0,
 (267, 3421): 0,
 (267, 3373): 0,
 (267, 392): 0,
 (267, 3500): 0,
 (267, 3407): 0,
 (267, 3329): 0,
 (267, 3335): 0,
 (267, 3387): 0,
 (267, 3315): 0,
 (267, 3396): 0,
 (267, 238): 0,
 (267, 3472): 0,
 (267, 297): 0,
 (267, 315): 0,
 (267, 3395): 0,
 (267, 3093): 0,
 (267, 3311): 0,
 (267, 260): 0,
 (267, 3105): 0,
 (267, 3457): 0,
 (267, 3340): 0,
 (267, 3073): 0,
 (267, 3422): 0,
 (267, 3067): 0,
 (267, 363): 0,
 (267, 3379): 0,
 (267, 3410): 0,
 (267, 3391): 0,
 (267, 3346): 0,
 (267, 3392): 0,
 (267, 3347): 0,
 (267, 3352): 0,
 (267, 3364): 0,
 (267, 3371): 0,
 (267, 3413): 0,
 (267, 3497): 0,
 (267, 3317): 0,
 (267, 3312): 0,
 (267, 3109): 0,
 (267, 3182): 0,
 (267, 343): 0,
 (267, 3409): 0,
 (267, 355): 0,
 (267, 3355): 0,
 (267, 325): 0,
 (267, 41

In [31]:
data_low.head(3)

,dock_id,dock_name,latitude,longitude,avail_bikes,tot_docks,avail_bikes_proportion,deficit
24,267,Broadway & W 36 St,40.750977,-73.987654,1,57,0.017544,18
145,3146,E 81 St & 3 Ave,40.775730,-73.956753,0,47,0.000000,16
79,469,Broadway & W 53 St,40.763441,-73.982681,3,57,0.052632,16


In [32]:
data_high.head(3)

,dock_id,dock_name,latitude,longitude,avail_bikes,tot_docks,avail_bikes_proportion,surplus
42,336,Sullivan St & Washington Sq,40.730477,-73.999061,36,36,1.000000,12
137,3132,E 59 St & Madison Ave,40.763505,-73.971092,34,35,0.971429,11
164,3224,W 13 St & Hudson St,40.739974,-74.005139,37,39,0.948718,11


In [61]:
data_low

,dock_id,dock_name,latitude,longitude,avail_bikes,tot_docks,avail_bikes_proportion,deficit
24,267,Broadway & W 36 St,40.750977,-73.987654,1,57,0.017544,18
145,3146,E 81 St & 3 Ave,40.775730,-73.956753,0,47,0.000000,16
79,469,Broadway & W 53 St,40.763441,-73.982681,3,57,0.052632,16
19,253,W 13 St & 5 Ave,40.735439,-73.994539,2,55,0.036364,16
8,168,W 18 St & 6 Ave,40.739713,-73.994564,1,47,0.021277,15
...,...,...,...,...,...,...,...,...
263,3399,7 St & 3 Ave,40.672603,-73.989830,7,24,0.291667,1
32,310,State St & Smith St,40.689269,-73.989129,12,36,0.333333,0
99,3059,Pulaski St & Marcus Garvey Blvd,40.693398,-73.939877,6,19,0.315789,0
275,3416,7 Ave & Park Pl,40.677615,-73.973243,8,25,0.320000,0
